In [1]:
import csv
import pandas as pd
import folium
from parse_arduino_data import parse_adafruit_latlon
import seaborn as sns
import glob
sns.set(style='white')

In [2]:
lines = []
with open('data/principal-shape.nmea', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        rowtype = row[0]
        if rowtype == '$GPGGA':
            rowtype, utc, lat, lat_dir, lon, lon_dir, quality, nsats, hdop, alt, a_units, undulation, u_units, *diff_data, check = row
            assert rowtype == '$GPGGA'
            assert a_units == u_units == 'M'
            line = [utc, str(lat) + lat_dir, str(lon) + lon_dir, quality, nsats, hdop, alt]
        elif rowtype == '$GPRMC':
            rowtype, utc, status, lat, lat_dir, lon, lon_dir, speed, track_true, date, *stuff, check = row
            assert rowtype == '$GPRMC'
            line.extend([status, speed, date])
            lines.append(line)
            line = None

In [3]:
df = pd.DataFrame(lines, columns=['utc', 'lat', 'lon', 'quality', 'nsats', 'hdop', 'alt', 'status', 'speed', 'date'])
df.query('status == "A"', inplace=True)
df['lat'] = df['lat'].map(parse_adafruit_latlon)
df['lon'] = df['lon'].map(parse_adafruit_latlon)
df['hdop'] = df['hdop'].astype(float)
df.query('hdop < 2', inplace=True)

In [4]:
m = folium.Map(
    tiles="Stadia.AlidadeSatellite",
    location=[18.448501, -66.592225],
    zoom_start=16,
    zoom_control=False,
)
for lat, lon in df[['lat', 'lon']].values:
    folium.CircleMarker(
        location=[lat, lon],
        radius=3,
        fill_opacity=1,
        fill=True,
        stroke=False,
        color='purple',
    ).add_to(m)
for pattern, color in [('data/*above*.csv', 'blue'), ('data/*below*.csv', 'green')]:
    for file in glob.glob(pattern):
        for lat, lon in pd.read_csv(file).query('hdop < 2')[['lat', 'lon']].values:
            folium.CircleMarker(
                location=[lat, lon],
                radius=3,
                fill_opacity=1,
                fill=True,
                stroke=False,
                color=color,
            ).add_to(m)
m